# Heroes of Pymoli

In [1]:
# import pandas
import pandas as pd

In [2]:
# Load the file

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

### Player Count

In [3]:
# Count the number of Players (which is the number of rows in dataset)
totalPlayers = purchase_data["SN"].value_counts().count()
# Display it as data frame
totalPlayersDF = pd.DataFrame({"Total Players":[ totalPlayers]})
totalPlayersDF

,Total Players
0,576


### Purchasing Analysis (Total)

In [4]:
# Number of unique items
totalUniqueItems = purchase_data["Item Name"].value_counts().count()
# Average purchase price
averagePurchasePrice = round(purchase_data["Price"].mean(),2)
# Total number of Purchases
totalNumberPurchases = purchase_data["Purchase ID"].count()
# Total Revenue
totalRevenue = purchase_data["Price"].sum()
PurchaseResults = pd.DataFrame({
    "Number of Unique Items":[totalUniqueItems],
    "Average Price": [averagePurchasePrice],
    "Number of Purchases": [totalNumberPurchases],
    "Total Revenue":[totalRevenue]
    
})
PurchaseResults["Average Price"] = PurchaseResults["Average Price"].map("${:.2f}".format)
PurchaseResults["Total Revenue"] = PurchaseResults["Total Revenue"].map("${:,.2f}".format)
PurchaseResults

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


### Gender Demographics

In [5]:
# Store unique players and their related info, (SN, Age, Gender) into dataframe
PlayerInfo = purchase_data.iloc[:,1:4]
uniquePlayersInfo = PlayerInfo.drop_duplicates(["SN","Gender"])
uniquePlayersInfoG = uniquePlayersInfo.groupby("Gender")
# #Find the counts of each gender

genderCounts = uniquePlayersInfoG.count()
genderCounts = pd.DataFrame(genderCounts["SN"])
femaleCount = genderCounts.loc["Female"]
maleCount = genderCounts.loc["Male"]
otherCount = genderCounts.loc["Other / Non-Disclosed"]
# Store each gender count into its own variable
# maleCount = genderCounts[0]
# femaleCount = genderCounts[1]
# otherCount = genderCounts[2]

malePercent = round((maleCount / totalPlayers) * 100,2)
femalePercent = round((femaleCount / totalPlayers) * 100,2)
otherPercent = round((otherCount / totalPlayers) * 100,2)

genderResults = pd.DataFrame({
    "Total Count": [maleCount[0], femaleCount[0], otherCount[0]],
    "Percentage of Players" : [malePercent[0],femalePercent[0], otherPercent[0]],
    "Gender" : ["Male", "Female","Other / Non-Disclosed"]
})

genderResults["Percentage of Players"] = genderResults["Percentage of Players"].map("{:.2f}%".format)
genderResults = genderResults.set_index("Gender")
genderResults

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


### Purchasing Analysis (Gender)

In [6]:
# Group by Gender
# Count how many purchases each gender made
purchGend = purchase_data[["Gender","SN"]].groupby("Gender").count()

# Rename SN column to Purchase Count
purchGend = purchGend.rename(columns = {"SN": "Purchase Count"})

# Average purchase price each for gender
purchGend["Average Purchase Price"] =purchase_data[["Gender","Price"]].groupby("Gender").mean()

# Add Total purchase price for each gender
purchGend["Total Purchase Value"] = purchase_data[["Gender","Price"]].groupby("Gender").sum()
purchGend

# Add Avergae total purchase per person
# Group by gender with name and price 
groupGender = purchase_data.groupby(["Gender","SN"])["Price"]
# Average Total Purchae per person for each gender
# Female
femaleMean = groupGender.sum().loc["Female"].mean()
# Male
maleMean = groupGender.sum().loc["Male"].mean()
# Other
othermean = groupGender.sum().loc["Other / Non-Disclosed"].mean()

purchGend["Avg Total Purrchase per Person"] = [femaleMean,maleMean,othermean]

# Format Data Frame
purchGend["Average Purchase Price"]= purchGend["Average Purchase Price"].map("${:,.2f}".format)
purchGend["Total Purchase Value"]= purchGend["Total Purchase Value"].map("${:,.2f}".format)
purchGend["Avg Total Purrchase per Person"]= purchGend["Avg Total Purrchase per Person"].map("${:,.2f}".format)

# Dispay Summary
purchGend

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purrchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### Age Demographics

In [7]:
# Remove duplicate player names
playerUnique = purchase_data.drop_duplicates(["SN","Age"])


In [8]:
# create bins
bins = [0,9,14,19,24,29,34,39,50]
# create labels
binLabels = ["<10", "10-14", "15-19","20-24","25-29","30-34","35-39","40+"]
#Place data into bins
binData = pd.cut(playerUnique["Age"],bins, labels = binLabels)
binData.value_counts()
# # add this column to dataframe
binData = pd.Series(binData, name ="Age Bracket")

ageBracket = pd.concat([playerUnique,binData],axis = 1)

# ageBracket[ageBracket["Age Bracket"].isnull()]
result = pd.DataFrame(ageBracket["Age Bracket"].value_counts()).sort_index()
result["Percentage of Players"] = (result["Age Bracket"] / totalPlayers)*100
result["Percentage of Players"] = result["Percentage of Players"].map("{:.2f}%".format)
result

,Age Bracket,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


### Purchasing Analysis (Age)

In [60]:
binData = pd.cut(purchase_data["Age"],bins, labels = binLabels)
purchase_data["Age Bracket"] = binData
# Total Count
purchase_data.groupby("Age Bracket")["SN"].count()

# Average Purchase Price
purchase_data.groupby("Age Bracket")["Price"].mean()

# Total Purchase value
purchase_data.groupby("Age Bracket")["Price"].sum()


# Average Total Purchase Person
groupAge = purchase_data.groupby(["Age Bracket","SN"])["Price"]
# Average Total Purchae per person for each Age Bracket

#THIS DOESNT WORK!! FIX
# Female
g1 = groupAge.sum().loc["<10"].sum()/23
# Male
g2 = groupAge.sum().loc["10-14"].mean()
# Other
g3 = groupAge.sum().loc["15-19"].mean()


3.353478260869565

In [34]:
23+28+136+365+101+73+41+13

780

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
Age Bracket    576
dtype: int64